In [29]:
# Importar las librerías necesarias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [30]:

# 1. Carga del Conjunto de Datos
# ==============================================================================

# URL proporcionada por el usuario
url = "https://hybridge-education-machine-learning-datasets.s3.us-east-1.amazonaws.com/Fraud.csv"
print(f"Descargando datos desde: {url}")

try:
    # Cargar el dataset en un DataFrame de pandas
    df = pd.read_csv(url)
    print("Carga de Datos: Exitosa ✅")
except Exception as e:
    print(f"Error al cargar los datos desde la URL. Verifica la conexión o la URL. Error: {e}")
    # Si la carga falla, se detiene la ejecución del laboratorio
    exit()

Descargando datos desde: https://hybridge-education-machine-learning-datasets.s3.us-east-1.amazonaws.com/Fraud.csv
Carga de Datos: Exitosa ✅


In [31]:
# 2. Exploración de Datos (Data Exploration)
# ==============================================================================

print("\n--- 2. Exploración de Datos ---")
print("Primeras 5 filas del dataset:")
print(df.head())
print("\nInformación de las columnas y tipos de datos:")
df.info()


--- 2. Exploración de Datos ---
Primeras 5 filas del dataset:
   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  

Información de las columnas y tipos de da

In [32]:
len(df)

6362620

# ¡¡¡6,362,620 OBSERVACIONES!!!

In [33]:
# Verificar la distribución de la variable objetivo (isFraud)
# El desbalance de clases es crítico en este tipo de problema.
clase_fraude = df['isFraud'].value_counts()
print("\nDistribución de la variable objetivo 'isFraud':")
print(clase_fraude)
print(f"Porcentaje de Fraude: {clase_fraude[1] / clase_fraude.sum() * 100:.4f}%")
print("El dataset está extremadamente desbalanceado.")

# Columnas con valores únicos (para identificar categóricas y columnas a descartar)
print("\nValores únicos en columnas categóricas clave:")
print(df['type'].unique())
print(f"Valores únicos en 'nameOrig': {df['nameOrig'].nunique()} (¡Demasiados, descartar!)")
print(f"Valores únicos en 'nameDest': {df['nameDest'].nunique()} (¡Demasiados, descartar!)")



Distribución de la variable objetivo 'isFraud':
isFraud
0    6354407
1       8213
Name: count, dtype: int64
Porcentaje de Fraude: 0.1291%
El dataset está extremadamente desbalanceado.

Valores únicos en columnas categóricas clave:
['PAYMENT' 'TRANSFER' 'CASH_OUT' 'DEBIT' 'CASH_IN']
Valores únicos en 'nameOrig': 6353307 (¡Demasiados, descartar!)
Valores únicos en 'nameDest': 2722362 (¡Demasiados, descartar!)


In [34]:

# 3. Preprocesamiento de Datos (Data Preprocessing)
# ==============================================================================

print("\n--- 3. Preprocesamiento de Datos ---")

# La columna 'isFlaggedFraud' es una bandera de simulación y no una característica de ML.
# Las columnas 'nameOrig' y 'nameDest' tienen demasiados valores únicos (alta cardinalidad) 
# y son difíciles de usar directamente en Regresión Logística. Se descartan.
X = df.drop(columns=['isFraud', 'isFlaggedFraud', 'nameOrig', 'nameDest'])
y = df['isFraud']

# Definir columnas numéricas y categóricas restantes
# 'step' y 'amount' son numéricas. Las columnas 'oldbalanceOrg', 'newbalanceOrg', etc. también lo son.
# 'type' es la única categórica significativa restante.
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist() # Debería ser solo ['type']

# Crear el Preprocesador (usando ColumnTransformer y Pipeline)
# 1. Escalar características numéricas (StandardScaler)
# 2. Codificar características categóricas (OneHotEncoder)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Mantiene cualquier otra columna (aunque no debería haber)
)

# Dividir el conjunto de datos
# Usar 'stratify=y' es esencial para mantener la proporción de fraude en ambos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Tamaño de Entrenamiento: {len(X_train)} filas")
print(f"Tamaño de Prueba: {len(X_test)} filas")



--- 3. Preprocesamiento de Datos ---
Tamaño de Entrenamiento: 5090096 filas
Tamaño de Prueba: 1272524 filas


In [35]:
#4. Entrenamiento del Modelo (Model Training)
# ==============================================================================

print("\n--- 4. Entrenamiento del Modelo ---")

# Crear el Pipeline final: Preprocesamiento -> Regresión Logística
# Se usa 'class_weight='balanced'' para mitigar el extremo desbalance de clases,
# dando más peso a la clase minoritaria (fraude).
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced'))
])

# Entrenar el modelo
model_pipeline.fit(X_train, y_train)
print("Modelo de Regresión Logística entrenado exitosamente.")
print("El entrenamiento ha compensado el desbalance de clases.")


--- 4. Entrenamiento del Modelo ---
Modelo de Regresión Logística entrenado exitosamente.
El entrenamiento ha compensado el desbalance de clases.


In [36]:
# 5. Evaluación del Desempeño (Performance Evaluation)
# ==============================================================================

print("\n--- 5. Evaluación del Desempeño ---")

# Predicciones
y_pred_test = model_pipeline.predict(X_test)

# Reporte de Clasificación (Precision, Recall, F1-score)
print("\nReporte de Clasificación (Conjunto de Prueba):")
print(classification_report(y_test, y_pred_test, target_names=['Legítima (0)', 'Fraudulenta (1)']))

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred_test)
print("\nMatriz de Confusión (para la clase Fraudulenta - 1):")
print(cm)

# Análisis de la Matriz de Confusión para Detección de Fraude
print("\nAnálisis de Desempeño (Clase Fraudulenta - 1):")
VP = cm[1, 1]  # Verdaderos Positivos: Fraudes detectados correctamente
FN = cm[1, 0]  # Falsos Negativos: Fraudes no detectados (el error más costoso)
FP = cm[0, 1]  # Falsos Positivos: Legítimas marcadas como fraude (falsas alarmas)

print(f"Verdaderos Positivos (VP): {VP}")
print(f"Falsos Negativos (FN): {FN}")
print(f"Falsos Positivos (FP): {FP}")

# El **Recall** (Sensibilidad) es la métrica más importante para la detección de fraude, 
# ya que mide la capacidad del modelo para capturar la mayoría de los casos de fraude real.
try:
    recall_fraude = VP / (VP + FN)
    print(f"\nRecall para la clase Fraudulenta (VP / (VP + FN)): {recall_fraude:.4f}")
except ZeroDivisionError:
    print("\nNo se pudieron calcular métricas de fraude. Verifica los datos.")

# Mostrar la Matriz de Confusión de forma visual (solo para referencia)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Legítima', 'Fraudulenta'])
# disp.plot(cmap=plt.cm.Blues)
# plt.title("Matriz de Confusión de Regresión Logística")
# plt.show()

# ==============================================================================
# Fin del Laboratorio
# ==============================================================================


--- 5. Evaluación del Desempeño ---

Reporte de Clasificación (Conjunto de Prueba):
                 precision    recall  f1-score   support

   Legítima (0)       1.00      0.95      0.97   1270881
Fraudulenta (1)       0.02      0.96      0.05      1643

       accuracy                           0.95   1272524
      macro avg       0.51      0.96      0.51   1272524
   weighted avg       1.00      0.95      0.97   1272524


Matriz de Confusión (para la clase Fraudulenta - 1):
[[1206584   64297]
 [     64    1579]]

Análisis de Desempeño (Clase Fraudulenta - 1):
Verdaderos Positivos (VP): 1579
Falsos Negativos (FN): 64
Falsos Positivos (FP): 64297

Recall para la clase Fraudulenta (VP / (VP + FN)): 0.9610
